In [1]:
import armageddon as ar
import numpy as np
import pandas as pd
earth = ar.Planet()

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

def plot(t, v, m, theta, z, x, r):
    energy = 0.5*m*v**2
    energy_z = energy.diff()/z.diff()
    fig = plt.figure(figsize=(15,12))
    plt.subplot(331)
    plt.plot(t,v) # v-t
    plt.title("v-t")
    plt.subplot(332)
    plt.plot(t,m) # m-t
    plt.title("m-t")
    plt.subplot(333)
    plt.plot(t,theta) # theta-t
    plt.title("$theta-t$")
    plt.subplot(334)
    plt.plot(t,z) # z-t
    plt.axhline(y=0,ls=":",c="r")
    plt.title("z-t")
    plt.subplot(335)
    plt.plot(t,x) # x-t
    plt.title("x-t")
    plt.subplot(336)
    plt.plot(t,r) # r-t
    plt.title("r-t")
    plt.subplot(337)
    plt.plot(z,v) # v-z
    plt.title("v-z")
    plt.subplot(338)
    plt.plot(energy_z/(4.184E12), z) # Energy-z
    plt.title("Energy-z")
    plt.subplot(339)
    plt.plot(x,z) # z-x
    plt.title("Trace")
    plt.show()

In [3]:
from scipy.integrate import odeint

def solve_atmospheric_odeint_entry(radius, velocity, density, strength, angle, init_altitude=100e3, dt=0.05, radians=False):
    if not radians:
        angle = angle * np.pi / 180
    t = np.arange(0, 500+dt, dt)
    init_state = (velocity, density * 4 / 3 * np.pi * (radius**3),
                angle, init_altitude, 0., radius)
    P1 = odeint(earth.dmove_odeint, init_state, t, args=([strength, density],))
    result_df = pd.DataFrame({'velocity': P1[:, 0], 'mass': P1[:, 1], 'angle': P1[:, 2]*180/np.pi,
                              'altitude': P1[:, 3], 'distance': P1[:, 4], 'radius': P1[:, 5],
                              'time': t}, index=range(len(P1)))
    return result_df

In [4]:
def diff(result1, result2):
    # absolute(a - b) <= (atol + rtol * absolute(b))
    # rtol=1e-05, atol=1e-08
    for col in result1.columns:
        same = np.allclose(result1[col], result2[col])
        v = np.sum(result1[col] - result2[col]) / len(result1[col])
        print("%10s\tdiff:%.10f\tAccepted:%s"  % (col, v, same))

In [5]:
input_data1 = {'radius': 1.,
              'velocity': 1.0e5,
              'density': 3000.,
              'strength': 1e32,
              'angle': 30.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data1)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data1)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

G:\ACSE\acse-4\project2\acse-4-armageddon-didymos\armageddon\solver.py:434: RuntimeWarning: invalid value encountered in double_scalars
  f[0] = -(self.Cd*rhoa*A*(u[0])**2)/(2*(u[1])) + self.g*np.sin(u[2])
G:\ACSE\acse-4\project2\acse-4-armageddon-didymos\armageddon\solver.py:434: RuntimeWarning: invalid value encountered in sin
  f[0] = -(self.Cd*rhoa*A*(u[0])**2)/(2*(u[1])) + self.g*np.sin(u[2])
G:\ACSE\acse-4\project2\acse-4-armageddon-didymos\armageddon\solver.py:435: RuntimeWarning: invalid value encountered in double_scalars
  f[1] = -(self.Ch*rhoa*A*(u[0])**3)/(2*self.Q)
G:\ACSE\acse-4\project2\acse-4-armageddon-didymos\armageddon\solver.py:436: RuntimeWarning: invalid value encountered in cos
  f[2] = self.g*np.cos(u[2])/(u[0]) - self.Cl*rhoa*A*(u[0])/(2*(u[1])) - u[0]*np.cos(u[2])/(self.Rp+u[3])
G:\ACSE\acse-4\project2\acse-4-armageddon-didymos\armageddon\solver.py:436: RuntimeWarning: invalid value encountered in double_scalars
  f[2] = self.g*np.cos(u[2])/(u[0]) - self.Cl*rh

  velocity	diff:71828612495155788804397872640497413601099776.0000000000	Accepted:False
      mass	diff:-1210635071174578028931547624277171458718870213206601884551852263886426779488667382980258189989150183205954918027362304.0000000000	Accepted:False
     angle	diff:98999553715162144.0000000000	Accepted:False
  altitude	diff:-3350674913348568481792.0000000000	Accepted:False
  distance	diff:-3140277324496240640000.0000000000	Accepted:False
    radius	diff:inf	Accepted:False
      time	diff:0.0000000000	Accepted:True


C:\Users\whynot\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [6]:
input_data2 = {'radius': 10.,
              'velocity': 21e3,
              'density': 3000.,
              'strength': 1e5,
              'angle': 45.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data2)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data2)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0240277412	Accepted:False
      mass	diff:-20.9392586546	Accepted:False
     angle	diff:0.0003052005	Accepted:False
  altitude	diff:8.2905119298	Accepted:False
  distance	diff:-6.7113923306	Accepted:False
    radius	diff:0.0693441280	Accepted:False
      time	diff:0.0000000000	Accepted:True


In [7]:
input_data3 = {'radius': 5.,
              'velocity': 11e3,
              'density': 3000.,
              'strength': 1e7,
              'angle': 60.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data3)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data3)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:0.2113345986	Accepted:False
      mass	diff:-16.1806990484	Accepted:False
     angle	diff:-0.0063701181	Accepted:False
  altitude	diff:-79.3627330857	Accepted:False
  distance	diff:25.4619993409	Accepted:False
    radius	diff:-0.2331391234	Accepted:False
      time	diff:0.0000000000	Accepted:True


In [8]:
input_data4 = {'radius': 15.,
              'velocity': 4e3,
              'density': 3000.,
              'strength': 1e7,
              'angle': 65.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data4)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data4)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:0.5092650839	Accepted:False
      mass	diff:-663.5436007990	Accepted:False
     angle	diff:-0.0087460555	Accepted:False
  altitude	diff:-201.9930245426	Accepted:False
  distance	diff:69.6166774390	Accepted:False
    radius	diff:-1.0571556264	Accepted:False
      time	diff:0.0000000000	Accepted:True


In [9]:
input_data5 = {'radius': 15.,
              'velocity': 16e3,
              'density': 3000.,
              'strength': 1e10,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data5)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data5)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000008733	Accepted:True
      mass	diff:0.3376021265	Accepted:True
     angle	diff:0.0000002952	Accepted:True
  altitude	diff:-0.0005836034	Accepted:False
  distance	diff:0.0030578887	Accepted:True
    radius	diff:0.0000000000	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [10]:
input_data6 = {'radius': 15.,
              'velocity': 16e3,
              'density': 3000.,
              'strength': 1e2,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data6)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data6)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000035989	Accepted:True
      mass	diff:-1.3436324095	Accepted:True
     angle	diff:0.0000001773	Accepted:True
  altitude	diff:0.0007005316	Accepted:True
  distance	diff:-0.0010098399	Accepted:True
    radius	diff:-0.0000084170	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [11]:
input_data7 = {'radius': 15.,
              'velocity': 16e3,
              'density': 3000.,
              'strength': 1e20,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data7)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data7)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000008733	Accepted:True
      mass	diff:0.3376021265	Accepted:True
     angle	diff:0.0000002952	Accepted:True
  altitude	diff:-0.0005836034	Accepted:False
  distance	diff:0.0030578887	Accepted:True
    radius	diff:0.0000000000	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [12]:
input_data8 = {'radius': 15.,
              'velocity': 16e3,
              'density': 3000.,
              'strength': 1e2,
              'angle': 45.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data8)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data8)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000055169	Accepted:True
      mass	diff:-6.5006270261	Accepted:True
     angle	diff:0.0000001966	Accepted:True
  altitude	diff:0.0018918296	Accepted:False
  distance	diff:-0.0016955530	Accepted:True
    radius	diff:-0.0000395266	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [13]:
input_data9 = {'radius': 15.,
              'velocity': 20e3,
              'density': 3000.,
              'strength': 1e2,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data9)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data9)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000072067	Accepted:True
      mass	diff:-1.6772816835	Accepted:True
     angle	diff:0.0000002823	Accepted:True
  altitude	diff:0.0013518413	Accepted:True
  distance	diff:-0.0024997011	Accepted:True
    radius	diff:-0.0000184639	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [14]:
input_data10 = {'radius': 15.,
              'velocity': 10e3,
              'density': 3000.,
              'strength': 1e2,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data10)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data10)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000020629	Accepted:True
      mass	diff:-0.0036788803	Accepted:True
     angle	diff:0.0000001174	Accepted:True
  altitude	diff:-0.0000569962	Accepted:False
  distance	diff:-0.0000674309	Accepted:True
    radius	diff:0.0000014304	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [15]:
input_data11 = {'radius': 10.,
              'velocity': 16e3,
              'density': 3000.,
              'strength': 1e2,
              'angle': 20.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data11)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data11)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000034102	Accepted:True
      mass	diff:-0.3072656646	Accepted:True
     angle	diff:0.0000001161	Accepted:True
  altitude	diff:0.0006016995	Accepted:True
  distance	diff:-0.0006739047	Accepted:True
    radius	diff:-0.0000044764	Accepted:True
      time	diff:0.0000000000	Accepted:True


In [24]:
input_data12 = {'radius': 10.,
              'velocity': 21e3,
              'density': 3000.,
              'strength': 1e3,
              'angle': 45.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data12)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data12)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:0.0014611494	Accepted:False
      mass	diff:-7.4368007837	Accepted:True
     angle	diff:-0.0000241469	Accepted:True
  altitude	diff:-0.4951970394	Accepted:False
  distance	diff:0.0224324135	Accepted:True
    radius	diff:-0.0079950538	Accepted:False
      time	diff:0.0000000000	Accepted:True


In [29]:
input_data13 = {'radius': 10.,
              'velocity': 21e3,
              'density': 3000.,
              'strength': 1e18,
              'angle': 45.0,
              'init_altitude':100e3,
              'dt': 0.05,
              'radians': False }
# numerical
result1 = earth.solve_atmospheric_entry(**input_data13)
# plot(result1.time, result1.velocity, result1.mass, result1.angle, result1.altitude, result1.distance, result1.radius)
# scipy_odeint
result2 = solve_atmospheric_odeint_entry(**input_data13)
# plot(result2.time, result2.velocity, result2.mass, result2.angle, result2.altitude, result2.distance, result2.radius)
diff(result1, result2)

  velocity	diff:-0.0000124460	Accepted:True
      mass	diff:-0.6873679761	Accepted:True
     angle	diff:0.0000003567	Accepted:True
  altitude	diff:0.0032205026	Accepted:False
  distance	diff:-0.0022908955	Accepted:True
    radius	diff:0.0000000000	Accepted:True
      time	diff:0.0000000000	Accepted:True
